# DAY1

In [1]:
import torch
import torchvision
import torchvision.datasets as Datasets
import torch.nn as nn
import torch.autograd as AG

In [2]:
torch.manual_seed(777)
train_data = Datasets.MNIST(root='MNIST_data/',
                           train = True,
                           transform=torchvision.transforms.ToTensor(),
                           download=True)
test_data = Datasets.MNIST(root='MNIST_data/',
                          train=False,
                          transform=torchvision.transforms.ToTensor(),
                          download=True)

In [3]:
def data_loader(data_set,batch_size):
    dataloader = torch.utils.data.DataLoader(dataset=data_set,batch_size=batch_size,shuffle=True)
    return dataloader

In [4]:
train_loader = data_loader(train_data,100)

In [5]:
# for getting image details
train_image,train_target = train_data[0]
train_image.shape

torch.Size([1, 28, 28])

In [5]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN,self).__init__()
        #Input channels = 1, output channels = 32 
        self.layer1 = nn.Sequential(
            nn.Conv2d(in_channels = 1,out_channels = 32,kernel_size = 3 ,stride = 1 ,padding = 1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2,stride=2))
        self.layer2 = nn.Sequential(
            nn.Conv2d(in_channels=32,out_channels=64,kernel_size=3,stride=1,padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2,stride=2))
        #Fully Connected Layer Next with 7x7x64 inputs to 10 outputs
        self.FC = nn.Linear(in_features=64*7*7,out_features=10,bias=True)
        nn.init.xavier_uniform_(self.FC.weight)
        
    def forward(self,x):
        layer1 = self.layer1(x)
        layer2 = self.layer2(layer1)
        layer2 = layer2.view(layer2.size(0),-1)
        out = self.FC(layer2)
        return out
        

In [6]:
# hyperparameters 
learning_rate = 0.001
training_epochs = 5
batch_size = 100
model = CNN()
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(),learning_rate)

# DAY2

In [7]:
import time

In [ ]:
# traing the model
training_time = time.time()

for epoch in range(training_epochs):
    print("Training of epoch {} started".format(epoch+1))
    avg_cost = 0
    total_batch = len(train_data)//batch_size # or len(train_loader)
    start_time = time.time()
    for i,(inputs,labels) in enumerate(train_loader):
        X = AG.Variable(inputs)
        Y = AG.Variable(labels)
        
        
        optimizer.zero_grad()
        pred = model(X)
        loss = criterion(pred,Y)
        loss.backward()
        optimizer.step()
        
        avg_cost+=loss.data / batch_size
    print("[Epoch: {:<4}] cost = {:_<4.3} took {}s".format(epoch + 1, avg_cost.item(),time.time()-start_time))

print('Learning Finished! in {}s'.format(time.time()-training_time))
    

Training of epoch 1 started
[Epoch: 1   ] cost = 1.34 took 103.65852069854736s
Training of epoch 2 started
[Epoch: 2   ] cost = 0.373 took 103.95655632019043s
Training of epoch 3 started
[Epoch: 3   ] cost = 0.269 took 102.42114639282227s
Training of epoch 4 started


In [ ]:
#test_loader = data_loader(test_data,len(test_data))

In [ ]:
#set the model to evaluation mode
model.eval()
X_test = AG.Variable(test_data.data.view(len(test_data),1,28,28).float())
Y_test = AG.Variable(test_data.targets)

#prediction time...
test_pred = model(X_test)
correct_pred = (torch.max(test_pred.data,dim=1)[1] == Y_test.data)
accuracy = correct_pred.float().mean().item()
print("Accuracy of the model is {}".format(accuracy*100))